In [1]:
from aoc_2023.gridthings import Grid, Node

text = """
FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
""".strip()

grid = Grid.from_string(text)
grid

<Grid (10, 20)>

In [2]:
for node in grid.flatten():
    if node.value == '-':
        grid.add_edge(node, node.left())
        grid.add_edge(node, node.right())
    elif node.value == '|':
        grid.add_edge(node, node.up())
        grid.add_edge(node, node.down())
    elif node.value == '7':
        grid.add_edge(node, node.left())
        grid.add_edge(node, node.down())
    elif node.value == 'J':
        grid.add_edge(node, node.up())
        grid.add_edge(node, node.left())
    elif node.value == 'L':
        grid.add_edge(node, node.right())
        grid.add_edge(node, node.up())
    elif node.value == 'F':
        grid.add_edge(node, node.down())
        grid.add_edge(node, node.right())

len(grid.edges)

196

In [3]:
for node in grid.flatten():
    if node.value == 'S':
        for neighbor in node.linear():
            if neighbor and node in grid.get_edges(neighbor):
                grid.add_edge(node, neighbor)
        break
node

<Node (4, 0) = S>

In [4]:
grid.get_edges(node)

{<Node (3, 0) = F>: 1, <Node (4, 1) = |>: 1}

In [5]:
nxt_node, _weight = grid.get_edges(node).popitem()
nxt_node

<Node (4, 1) = |>

In [6]:
path = [node, nxt_node]
while True:
    last_node = path[-2]
    current_node = path[-1]
    edges = list(grid.get_edges(current_node))
    nxt = edges[0] if edges[0] != last_node else edges[1]
    if nxt.value == 'S':
        break
    path.append(nxt)
        
len(path)

160

In [7]:
output = ''
for row in grid.rows():
    row_line = ''
    for node in row:
        if node in path:
            row_line += 'P'
        else:
            row_line += '.'
    np = row_line.count('P')
    output += row_line + ' ' + str(np) + '\n'

print(output)

.PPPPPPPPPPPPPPPPPPP 19
.PPPPPPPPPPPPPPPPPPP 19
.PPPPPPPPPPPPPPPPPP. 18
PPPPPPPPPPPPPP.PPPP. 18
PPPPPPPPPP....PPPP.. 14
...PPPPPPPP...PP.... 10
..PPPPPPPPPPP..PPPPP 16
..PPPPPPPPPPPPPPPPPP 18
.....PPPPPPPPPPPPPPP 15
.....PPPPPPPPPPPPP.. 13



In [8]:
path[-1], path[0], path[1]

(<Node (3, 0) = F>, <Node (4, 0) = S>, <Node (4, 1) = |>)

In [9]:
# Figure out what the path[0] value is supposed to be based on path[-1] and path[1].
# i.e. if they're the same row, then path[0] must be a horizontal connector ('-')
if path[1].row == path[-1].row: # same row, start connects horizontally
    path[0].value = '-'
elif path[1].col == path[-1].col: # same col, start connects vertically
    path[0].value = '|'
elif path[1].row > path[-1].row: # start path needs to route downwards
    if path[1].col > path[-1].col: # route is coming from the left
        path[0].value = '7'
    else:
        path[0].value = 'F'
else: # start path needs to route upwards
    if path[1].col > path[-1].col: # path is coming from the left
        path[0].value = 'J'
    else:
        path[0].value = 'L'

path[0].row, path[0].col, path[0].value

(0, 4, '7')

In [10]:
from typing import List

# Ray cast each point by looking for walls to the right. An odd number means its inside the polygon
# Note a few gotchas though:
#  1) '-' is in the path but doesn't count as a wall
#  2) when going up-right-up (F-J) that's technically only one wall
in_grid = []
for node in grid.flatten():
    if node in path:
        continue
    wall_count = 0
    last_turn = None
    ray: List[Node] = [n for n in grid.get_row(node.row) if n in path and n.col > node.col]
    for n in ray:
        if n.value == '-': # don't count horizontal as part of the polygon
            continue 
        if n.value == '|':
            wall_count += 1
        if n.value == 'F':
            wall_count += 1
            last_turn = 'F'
        if n.value == 'L':
            wall_count += 1
            last_turn = 'L'
        if n.value == '7':
            if last_turn == 'F':
                wall_count += 1
            last_turn = None
        if n.value == 'J':
            if last_turn == 'L':
                wall_count += 1
            last_turn = None
    if wall_count % 2 == 1:
        in_grid.append(node)
        
len(in_grid)      

10

In [11]:
output = ''
for row in grid.rows():
    row_line = ''
    for node in row:
        if node in path:
            # row_line += 'P'
            row_line += node.value
        elif node in in_grid:
            row_line += 'I'
        else:
            row_line += 'O'
    np = row_line.count('I')
    output += row_line + ' ' + str(np) + '\n'

print(output)

OF7F7F7F7F7F7F7F---7 0
O|LJ||||||||||||F--J 0
OL-7LJLJ||||||LJL-7O 0
F--JF--7||LJLJIF7FJO 1
L---JF-JLJIIIIFJLJOO 4
OOOF-JF---7IIIL7OOOO 3
OOFJF7L7F-JF7IIL---7 2
OOL-JL7||F7|L7F-7F7| 0
OOOOOFJ|||||FJL7||LJ 0
OOOOOL-JLJLJL--JLJOO 0



In [12]:
print(text)

FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L
